In [1]:
!nvidia-smi

Fri Jul 29 09:13:53 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install pinecone-client==2.0.8

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 149 kB 4.6 MB/s 
     |████████████████████████████████| 58 kB 8.1 MB/s 
     |████████████████████████████████| 596 kB 44.1 MB/s 
     |████████████████████████████████| 269 kB 68.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
!pip install -U 'farm-haystack[pinecone]'==1.3.0
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 475 kB 4.8 MB/s 
     |████████████████████████████████| 10.7 MB 61.1 MB/s 
     |████████████████████████████████| 3.3 MB 42.5 MB/s 
     |████████████████████████████████| 85 kB 5.8 MB/s 
     |████████████████████████████████| 14.1 MB 48.5 MB/s 
     |████████████████████████████████| 981 kB 13.1 MB/s 
     |████████████████████████████████| 381 kB 68.8 MB/s 
     |████████████████████████████████| 50 kB 8.6 MB/s 
     |████████████████████████████████| 5.6 MB 56.8 MB/s 
     |████████████████████████████████| 219 kB 70.1 MB/s 
     |████████████████████████████████| 2.0 MB 56.1 MB/s 
     |████████████████████████████████| 178 kB 76.7 MB/s 
     |████████████████████████████████| 321 kB 75.6 MB/s 
     |████████████████████████████████| 43 kB 2.6 MB/s 
     |██████████████████████████████▎ | 834.1 MB 1.2 MB/s eta 0:00:41tcmalloc: large alloc 114

In [15]:
from haystack.document_stores import PineconeDocumentStore

document_store = PineconeDocumentStore(
    api_key='<YOUR-API-KEY>'
    index='haystack-lfqa',
    similarity="cosine",
    embedding_dim=768
)

INFO - haystack.document_stores.pinecone -  Index statistics: name: haystack-lfqa, embedding dimensions: 768, record count: 0


In [16]:
document_store.metric_type

'cosine'

In [17]:
document_store.get_document_count()

0

In [18]:
from datasets import load_dataset

wiki_data = load_dataset(
    'vblagoje/wikipedia_snippets_streamed',
    split='train',
    streaming=True
)
wiki_data

WARNING - datasets.builder -  Using custom data configuration default


In [19]:
# show the contents of a single document in the dataset
next(iter(wiki_data))

{'article_title': "St John the Baptist's Church, Atherton",
 'end_character': 511,
 'end_paragraph': 6,
 'passage_text': "St John the Baptist's Church, Atherton History There have been three chapels or churches on the site of St John the Baptist parish church. The first chapel at Chowbent was built in 1645 by John Atherton as a chapel of ease of Leigh Parish Church. It was sometimes referred to as the Old Bent Chapel. It was not consecrated and used by the Presbyterians as well as the Vicar of Leigh. In 1721 Lord of the manor Richard Atherton expelled the dissenters who subsequently built Chowbent Chapel. The first chapel was consecrated in 1723 by the Bishop of Sodor and",
 'section_title': 'History',
 'start_character': 0,
 'start_paragraph': 2,
 'wiki_id': 'Q7593707'}

In [20]:
# Filter only documents with History as section_title
history = wiki_data.filter(lambda d: d['section_title'].startswith('History'))
history

In [21]:
from haystack import Document
from tqdm.auto import tqdm  # progress bar

total_doc_count = 10000
batch_size = 10000

counter = 0
docs = []
for d in tqdm(history, total=total_doc_count):
    # create haystack document object with text content and doc metadata
    doc = Document(
        content=d["passage_text"],
        meta={
            "article_title": d["article_title"],
            'section_title': d['section_title']
        }
    )
    docs.append(doc)
    counter += 1
    if counter % batch_size == 0:
        # writing docs everytime 10k docs are reached
        document_store.write_documents(docs)
        docs.clear()
    if counter == total_doc_count:
        break

  0%|          | 0/10000 [00:00<?, ?it/s]

Writing Documents:   0%|          | 0/10000 [00:00<?, ?it/s]

In [22]:
document_store.get_document_count()

10000

In [7]:
import torch
# confirm GPU is available (if using CPU this step will be slower)
torch.cuda.is_available()

True

In [8]:
from haystack.retriever.dense import EmbeddingRetriever

retriever = EmbeddingRetriever(
    document_store=document_store,
    embedding_model="flax-sentence-embeddings/all_datasets_v3_mpnet-base",
    model_format="sentence_transformers"
)

INFO - haystack.modeling.utils -  Using devices: CUDA:0
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.retriever.dense -  Init retriever using embeddings of model flax-sentence-embeddings/all_datasets_v3_mpnet-base


Downloading:   0%|          | 0.00/737 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.85k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/591 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/383 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

WARNING - haystack.nodes.retriever._embedding_encoder -  You are using a Sentence Transformer with the None function. We recommend using cosine instead. This can be set when initializing the DocumentStore


In [23]:
document_store.update_embeddings(
    retriever,
    batch_size=256
)

INFO - haystack.document_stores.pinecone -  Updating embeddings for 10000 docs...


Updating Embedding:   0%|          | 0/10000 [00:00<?, ? docs/s]

Batches:   0%|          | 0/8 [00:00<?, ?it/s]

Batches:   0%|          | 0/8 [00:00<?, ?it/s]

Batches:   0%|          | 0/8 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/8 [00:00<?, ?it/s]

Batches:   0%|          | 0/8 [00:00<?, ?it/s]

Batches:   0%|          | 0/8 [00:00<?, ?it/s]

Batches:   0%|          | 0/8 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [24]:
from haystack.pipelines import DocumentSearchPipeline
from haystack.utils import print_documents

search_pipe = DocumentSearchPipeline(retriever)
result = search_pipe.run(
    query="When was the first electric power system built?",
    params={"Retriever": {"top_k": 2}}
)

print_documents(result)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Query: When was the first electric power system built?

{   'content': 'Electric power system History In 1881, two electricians built '
               "the world's first power system at Godalming in England. It was "
               'powered by two waterwheels and produced an alternating current '
               'that in turn supplied seven Siemens arc lamps at 250 volts and '
               '34 incandescent lamps at 40 volts. However, supply to the '
               'lamps was intermittent and in 1882 Thomas Edison and his '
               'company, The Edison Electric Light Company, developed the '
               'first steam-powered electric power station on Pearl Street in '
               'New York City. The Pearl Street Station initially powered '
               'around 3,000 lamps for 59 customers. The power station '
               'generated direct current and',
    'name': None}

{   'content': 'consultant at the Westinghouse Electric & Manufacturing '
               "Company'

In [25]:
from haystack.generator.transformers import Seq2SeqGenerator

generator = Seq2SeqGenerator(model_name_or_path="vblagoje/bart_lfqa")

INFO - haystack.modeling.utils -  Using devices: CUDA
INFO - haystack.modeling.utils -  Number of GPUs: 1


In [26]:
from haystack.pipelines import GenerativeQAPipeline

pipe = GenerativeQAPipeline(generator, retriever)

In [27]:

from haystack.pipelines import GenerativeQAPipeline

pipe = GenerativeQAPipeline(generator, retriever)

In [28]:
result = pipe.run(
        query="what was the war of currents?",
        params={
            "Retriever": {"top_k": 3},
            "Generator": {"top_k": 1}
        })

result

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

{'answers': [<Answer {'answer': "The War of Currents was a rivalry between Edison and George Westinghouse's companies over which form of transmission (direct or alternating current) was superior. In 1891, the first major power system that was designed to drive a generator was installed in the United States and Europe. By 1888, the electric power industry was flourishing, and power companies had built thousands of power systems (both direct and alternating current). The rivalry between the two companies had grown into a propaganda campaign.", 'type': 'generative', 'score': None, 'context': None, 'offsets_in_document': None, 'offsets_in_context': None, 'document_id': None, 'meta': {'doc_ids': ['2a792dfd942f088b1f0a01f3a0470afd', '3a43249b33b1435e94ef9b22f01989b6', 'ef560b97430a71ba9ff193062d6a9d0b'], 'doc_scores': [0.5009062511226035, 0.5010571982595333, 0.5009244725015314], 'content': ["outbreak of war included 38 submarines.\nDuring the war the major operating arenas were the Norwegian

In [29]:
from haystack.utils import print_answers

result = pipe.run(
        query="what was the war of currents?",
        params={
            "Retriever": {"top_k": 3},
            "Generator": {"top_k": 1}
        })

print_answers(result, details="minimum")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Query: what was the war of currents?
Answers:
[   {   'answer': 'The War of Currents was a rivalry between Edison and George '
                  "Westinghouse's companies over which form of transmission "
                  '(direct or alternating current) was superior. In 1891, the '
                  'first major power system that was designed to drive a '
                  'generator was installed in the United States and Europe. By '
                  '1888, the electric power industry was flourishing, and '
                  'power companies had built thousands of power systems (both '
                  'direct and alternating current). The rivalry between the '
                  'two companies had grown into a propaganda campaign.'}]
